# Wrangle NHANES

Pulling raw NHANES dietary recall data for 2017-2020 and 2021-2023 waves, joining with FPED, 

## Set Working Directory

When you open a notebook, the default working directory will be the folder that notebook is in. We want it to be the top (root) directory of the project, `ds1_nhanes`.

First, we need to mount our Google Drive, which contains the `ds1_nhanes` folder. The following chunk will mount the drive (if in Google Colab) and set the working directory to the root of the project folder.

In [2]:
import os
import re

try:
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/ds1_nhanes/')
except:
  from pathlib import Path
  if not re.search(r'ds1_nhanes$', str(os.getcwd())):
    os.chdir(Path(os.getcwd()).parent)

print(os.getcwd())

c:\Users\d1033\Documents\git_repos\ds1_nhanes


In [3]:
import pandas as pd
import re

# str(os.getcwd()).str.contains('repos')
bool(re.search(r'repos$', str(os.getcwd())))

False

Bingo bongo, we're good to go.

## Load Data and Libraries

In [4]:
import pandas as pd
import numpy as np
import re

The DR1IFF_L and DR2IFF_L datasets include daily food intake over two days for respondents identified by the SEQN or respondent sequence identifier. The surveys were conducted in waves and we aim to combine two waves of data survey responses from 2017-2020 and 2021-2023. Given the size of these datasets we tried to pull only the columns that are relevant for our analysis are read for both days and both years, however, usecols does not work on .XPT files.

In [5]:
# Read in the four datasets with specified columns
dr1_17 = pd.read_sas('data/raw/nhanes_2017_2020/P_DR1IFF.xpt')
dr2_17 = pd.read_sas('data/raw/nhanes_2017_2020/P_DR2IFF.xpt')
dr1_21 = pd.read_sas('data/raw/nhanes_2021_2023/DR1IFF_L.xpt')
dr2_21 = pd.read_sas('data/raw/nhanes_2021_2023/DR2IFF_L.xpt')

# Create a list of all the datasets
datasets = [dr1_21, dr2_21, dr1_17, dr2_17]

# Define the relevant columns for this analysis
relevant_cols = ['SEQN', # response_sequence, unique identifier for each respondent
                 'WTDR2D', # weight_day_2_dietary, the weighting factor given to
                 # the second day depending on how many days the respondents reported
                 'WTDR2DPP', # different name in 2017 - 2020 data
                 'DR1IGRMS', # grams, total grams of food consumed, labeled DR1GRMS or DR2GRMS
                 # based on which day it was reported
                 'DR2IGRMS', # labeled DR1GRMS or DR2GRMS based on which day it was reported
                 'DR1IFDCD', # usda_food_code, food identifier
                 'DR2IFDCD'] # labeled DR1GRMS or DR2GRMS based on which day it was reported
for dataset in datasets:
  dataset.drop(columns=[col for col in dataset.columns if col not in relevant_cols], inplace=True)

#Look up what columns are relevant and load only what is relevant, check class notes - LB
print(dr1_21.info())
print(dr2_21.info())
print(dr1_17.info())
print(dr2_17.info())
dr1_21.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100116 entries, 0 to 100115
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   SEQN      100116 non-null  float64
 1   WTDR2D    100116 non-null  float64
 2   DR1IFDCD  100116 non-null  float64
 3   DR1IGRMS  99787 non-null   float64
dtypes: float64(4)
memory usage: 3.1 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88032 entries, 0 to 88031
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      88032 non-null  float64
 1   WTDR2D    88032 non-null  float64
 2   DR2IFDCD  88032 non-null  float64
 3   DR2IGRMS  87778 non-null  float64
dtypes: float64(4)
memory usage: 2.7 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183910 entries, 0 to 183909
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   SEQN      183910 non-nu

,SEQN,WTDR2D,DR1IFDCD,DR1IGRMS
0,130378.0,70554.222162,94000100.0,120.0
1,130378.0,70554.222162,94000100.0,120.0
2,130378.0,70554.222162,92101000.0,300.0
3,130378.0,70554.222162,94000100.0,240.0
4,130378.0,70554.222162,83102000.0,4.9


## Explore Dietary Recall Data

Compare the dimensions of the df with the number of unique SEQN numbers (respondent ids)

In [6]:
# Compare rows to unique respondent IDs
# first get number of rows
rows = dr1_21.shape[0]
unique_seqns = dr1_21['SEQN'].nunique()
print(f"{rows} rows and {unique_seqns} unique SEQN numbers in DR1.")

100116 rows and 6751 unique SEQN numbers in DR1.


There are far more rows than unique respondents. This is because for each respondent, there is one row for each individual food they consumed.

Check out how many unique food codes there are:

In [7]:
n_codes = dr1_21['DR1IFDCD'].nunique()
print(f"There are {n_codes} unique food codes")

There are 3987 unique food codes


In [8]:
# Check SEQNs between dr1 and dr2

diff = set(dr1_17['SEQN']).difference(dr2_17['SEQN'])
print(f"{len(diff)} SEQNs missing from DR2 that were in DR1 in the 2017 - 2020 wave")
diff2 = set(dr2_17['SEQN']).difference(dr1_17['SEQN'])
print(f"{len(diff2)} SEQNs missing from DR1 that were in DR2 in the 2017 - 2020 wave")

diff = set(dr1_21['SEQN']).difference(dr2_21['SEQN'])
print(f"{len(diff)} SEQNs missing from DR2 that were in DR1 in the 2021 - 2023 wave")
diff2 = set(dr2_21['SEQN']).difference(dr1_21['SEQN'])
print(f"{len(diff2)} SEQNs missing from DR1 that were in DR2 in the 2021 - 2023 wave")



1804 SEQNs missing from DR2 that were in DR1 in the 2017 - 2020 wave
2 SEQNs missing from DR1 that were in DR2 in the 2017 - 2020 wave
873 SEQNs missing from DR2 that were in DR1 in the 2021 - 2023 wave
1 SEQNs missing from DR1 that were in DR2 in the 2021 - 2023 wave


For the 2017 - 2020 wave, there are 1804 respondants for day 1 that did not report in day 2, and two respondent who reported in day 2 but not day 1.

For the 2021 - 2023 wave, there are 873 respondants for day 1 that did not report in day 2, and one respondent who reported in day 2 but not day 1.  people when we join.

The two-day weights (WTDR2) was adjusted based on the day 1 weights (WTDR1) and further adjusting for additional non-response for the second recall, so we will drop the respondents that only respond in both days.


## Join with FPED

Join FPED to each of the four DR datasets. We have to do this and aggregate within each DR before we combine the DRs together, otherwise we get cartesian merges.

In [9]:
fped = pd.read_csv('data/miscellany/FPED_1720.csv')
fped.columns = fped.columns.str.lower()
fped.columns = fped.columns.str.replace(" ", "_")

fped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7444 entries, 0 to 7443
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   foodcode                   7444 non-null   int64  
 1   description                7444 non-null   object 
 2   f_total_(cup_eq)           7444 non-null   float64
 3   f_citmlb_(cup_eq)          7444 non-null   float64
 4   f_other_(cup_eq)           7444 non-null   float64
 5   f_juice_(cup_eq)           7444 non-null   float64
 6   v_total_(cup_eq)           7444 non-null   float64
 7   v_drkgr_(cup_eq)           7444 non-null   float64
 8   v_redor_total_(cup_eq)     7444 non-null   float64
 9   v_redor_tomato_(cup_eq)    7444 non-null   float64
 10  v_redor_other_(cup_eq)     7444 non-null   float64
 11  v_starchy_total_(cup_eq)   7444 non-null   float64
 12  v_starchy_potato_(cup_eq)  7444 non-null   float64
 13  v_starchy_other_(cup_eq)   7444 non-null   float

Rename columns in DR datasets that are not consistent (FDCD, GRMS, and 2 day weights). This will make it easier map over the list of all datasets when we aggregate

In [10]:
# Function to rename columns across all datasets
# These are the ones we care about that differ between DR dataset
def rename_columns(df):
    new_columns = {}
    for col in df.columns:
        if re.search(r'FDCD$', col):
            new_columns[col] = 'food_code'
        elif re.search(r'GRMS$', col):
            new_columns[col] = 'grams'
        elif re.search(r'^WTDR2', col):
            new_columns[col] = 'weight_2d'
        else:
            new_columns[col] = col

    df = df.rename(columns=new_columns)
    return df

# Rename columns in each dataset
datasets_renamed = list(map(rename_columns, datasets))

print("\nRenamed Datasets:\n")
list(map(lambda df: df.info(), datasets_renamed))



Renamed Datasets:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100116 entries, 0 to 100115
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   SEQN       100116 non-null  float64
 1   weight_2d  100116 non-null  float64
 2   food_code  100116 non-null  float64
 3   grams      99787 non-null   float64
dtypes: float64(4)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88032 entries, 0 to 88031
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SEQN       88032 non-null  float64
 1   weight_2d  88032 non-null  float64
 2   food_code  88032 non-null  float64
 3   grams      87778 non-null  float64
dtypes: float64(4)
memory usage: 2.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183910 entries, 0 to 183909
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0  

[None, None, None, None]

Merge each DR with FPED

In [11]:
# Map over our list of datasets and merge each out with FPED
def merge_with_fped(df):
    return df.merge(fped, left_on='food_code', right_on='foodcode', how='left')

dfs_fped = list(map(merge_with_fped, datasets_renamed))

print("\nDatasets with FPED:\n")
list(map(lambda df: df.info(), dfs_fped))


Datasets with FPED:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100116 entries, 0 to 100115
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   SEQN                       100116 non-null  float64
 1   weight_2d                  100116 non-null  float64
 2   food_code                  100116 non-null  float64
 3   grams                      99787 non-null   float64
 4   foodcode                   99788 non-null   float64
 5   description                99788 non-null   object 
 6   f_total_(cup_eq)           99788 non-null   float64
 7   f_citmlb_(cup_eq)          99788 non-null   float64
 8   f_other_(cup_eq)           99788 non-null   float64
 9   f_juice_(cup_eq)           99788 non-null   float64
 10  v_total_(cup_eq)           99788 non-null   float64
 11  v_drkgr_(cup_eq)           99788 non-null   float64
 12  v_redor_total_(cup_eq)     99788 non-null   float64
 13  v_redor

[None, None, None, None]

## Combine Datasets

Now we can concat DR1 and DR2 together for each wave

In [12]:
# Combine first and second DF to make a df for 2021-2023
df_21 = pd.concat([dfs_fped[0], dfs_fped[1]], ignore_index=True)
df_21.info()

# Combine second and third DF to make df for 2017-2020
df_17 = pd.concat([dfs_fped[2], dfs_fped[3]], ignore_index=True)
df_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188148 entries, 0 to 188147
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   SEQN                       188148 non-null  float64
 1   weight_2d                  188148 non-null  float64
 2   food_code                  188148 non-null  float64
 3   grams                      187565 non-null  float64
 4   foodcode                   187510 non-null  float64
 5   description                187510 non-null  object 
 6   f_total_(cup_eq)           187510 non-null  float64
 7   f_citmlb_(cup_eq)          187510 non-null  float64
 8   f_other_(cup_eq)           187510 non-null  float64
 9   f_juice_(cup_eq)           187510 non-null  float64
 10  v_total_(cup_eq)           187510 non-null  float64
 11  v_drkgr_(cup_eq)           187510 non-null  float64
 12  v_redor_total_(cup_eq)     187510 non-null  float64
 13  v_redor_tomato_(cup_eq)    18

Get food group totals for each food code for each person. Take grams, divide by 100, then multiply by every food group category.

In [13]:
# Put them back into a list
waves = [df_17, df_21]

def get_food_group_totals(df):
    df.loc[:, 'f_total_(cup_eq)':'a_drinks_(no._of_drinks)'].multiply(df['grams']/100, axis=0)
    return df

waves_fped = list(map(
    get_food_group_totals,
    waves
))

waves_fped[0].head()

,SEQN,weight_2d,food_code,grams,foodcode,description,f_total_(cup_eq),f_citmlb_(cup_eq),f_other_(cup_eq),f_juice_(cup_eq),...,pf_nutsds_(oz_eq),pf_legumes_(oz_eq),d_total_(cup_eq),d_milk_(cup_eq),d_yogurt_(cup_eq),d_cheese_(cup_eq),oils_(grams),solid_fats_(grams),add_sugars_(tsp_eq),a_drinks_(no._of_drinks)
0,109263.0,17808.067666,28320300.0,199.5,28320300,"Pork with vegetable excluding carrots, broccol...",0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.0,0.0,0.81,0.14,0.00,0.0
1,109263.0,17808.067666,91746110.0,20.0,91746110,"Chocolate candy, candy shell with nuts",0.0,0.0,0.0,0.0,...,1.99,0.0,0.05,0.03,0.0,0.0,5.09,16.02,11.52,0.0
2,109263.0,17808.067666,58106210.0,238.0,58106210,"Pizza, cheese, from restaurant or fast food, N...",0.0,0.0,0.0,0.0,...,0.00,0.0,0.70,0.00,0.0,0.7,3.54,5.01,0.26,0.0
3,109263.0,17808.067666,64104010.0,209.0,64104010,"Apple juice, 100%",0.4,0.0,0.0,0.4,...,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0
4,109263.0,17808.067666,11710801.0,124.0,11710801,"Infant formula, ready-to-feed (PediaSure)",0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0


Group by SEQN and aggregate FPED variables

In [14]:
## Grouping by SEQN within each DR dataset
# Set aggregation functions so we don't have to do them all manually
# Everything except food code and description in FPED should be summedf
cols_to_sum = fped.columns[2:]

# Set aggregation functions
aggs = {col: 'sum' for col in cols_to_sum}
aggs['SEQN'] = 'first'
aggs['weight_2d'] = 'unique'
aggs['grams'] = 'sum'

# Aggregate each dataset
waves_grouped = list(map(
    lambda df:
        df.groupby('SEQN').agg(aggs),
    waves_fped
))

waves_grouped[0].head()

,f_total_(cup_eq),f_citmlb_(cup_eq),f_other_(cup_eq),f_juice_(cup_eq),v_total_(cup_eq),v_drkgr_(cup_eq),v_redor_total_(cup_eq),v_redor_tomato_(cup_eq),v_redor_other_(cup_eq),v_starchy_total_(cup_eq),...,d_milk_(cup_eq),d_yogurt_(cup_eq),d_cheese_(cup_eq),oils_(grams),solid_fats_(grams),add_sugars_(tsp_eq),a_drinks_(no._of_drinks),SEQN,weight_2d,grams
SEQN,,,,,,,,,,,,,,,,,,,,,
109263.0,1.60,0.00,0.00,1.60,0.36,0.00,0.15,0.15,0.00,0.00,...,0.26,0.00,3.19,25.41,82.28,49.04,0.0,109263.0,[17808.067666],2827.75
109264.0,0.00,0.00,0.00,0.00,4.17,0.86,1.03,0.66,0.37,0.00,...,0.12,0.00,3.09,119.77,107.89,43.70,0.0,109264.0,[7253.761719],4177.01
109265.0,2.28,0.67,0.41,1.20,3.34,0.21,0.38,0.16,0.22,1.93,...,1.43,0.00,2.35,104.86,168.54,45.45,0.0,109265.0,[35612.007356],4081.76
109266.0,0.87,0.70,0.17,0.00,3.98,0.07,0.60,0.16,0.44,2.48,...,0.20,0.26,2.44,82.41,40.39,14.80,0.0,109266.0,[5988.203624],9866.51
109269.0,0.04,0.00,0.00,0.04,1.85,0.00,0.10,0.10,0.00,1.75,...,0.25,0.00,2.99,175.65,87.97,18.82,0.0,109269.0,[18231.925894],1906.30


Now we are down to 1 row per SEQN per wave, including both days of dietary recall.

Finally we can concat both waves into a single df and divide 2day weights by 2


In [15]:
# Combine waves
df = pd.concat(waves_grouped, ignore_index=True)

# Divide 2day weights by 2 since we are combining two waves
df['weight_2d'] = df['weight_2d'] / 2

# Rearrange columns by bringing important ones to front
# Leaving in grams just as a check
front_cols = ['SEQN', 'weight_2d', 'grams']
cols = front_cols + [col for col in df.columns if col not in front_cols]
df = df[cols]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19386 entries, 0 to 19385
Data columns (total 40 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQN                       19386 non-null  float64
 1   weight_2d                  19386 non-null  object 
 2   grams                      19386 non-null  float64
 3   f_total_(cup_eq)           19386 non-null  float64
 4   f_citmlb_(cup_eq)          19386 non-null  float64
 5   f_other_(cup_eq)           19386 non-null  float64
 6   f_juice_(cup_eq)           19386 non-null  float64
 7   v_total_(cup_eq)           19386 non-null  float64
 8   v_drkgr_(cup_eq)           19386 non-null  float64
 9   v_redor_total_(cup_eq)     19386 non-null  float64
 10  v_redor_tomato_(cup_eq)    19386 non-null  float64
 11  v_redor_other_(cup_eq)     19386 non-null  float64
 12  v_starchy_total_(cup_eq)   19386 non-null  float64
 13  v_starchy_potato_(cup_eq)  19386 non-null  flo

In [16]:
# Make the SEQN column an integer and weight_2d a float with two decimal places

df['SEQN'] = df['SEQN'].astype(int)

df['weight_2d'] = df['weight_2d'].astype(float)
df['weight_2d'] = df['weight_2d'].round(2)

df

,SEQN,weight_2d,grams,f_total_(cup_eq),f_citmlb_(cup_eq),f_other_(cup_eq),f_juice_(cup_eq),v_total_(cup_eq),v_drkgr_(cup_eq),v_redor_total_(cup_eq),...,pf_nutsds_(oz_eq),pf_legumes_(oz_eq),d_total_(cup_eq),d_milk_(cup_eq),d_yogurt_(cup_eq),d_cheese_(cup_eq),oils_(grams),solid_fats_(grams),add_sugars_(tsp_eq),a_drinks_(no._of_drinks)
0,109263,8904.03,2827.75,1.60,0.00,0.00,1.60,0.36,0.00,0.15,...,1.99,0.00,3.47,0.26,0.00,3.19,25.41,82.28,49.04,0.00
1,109264,3626.88,4177.01,0.00,0.00,0.00,0.00,4.17,0.86,1.03,...,0.52,0.00,3.26,0.12,0.00,3.09,119.77,107.89,43.70,0.00
2,109265,17806.00,4081.76,2.28,0.67,0.41,1.20,3.34,0.21,0.38,...,0.00,0.00,3.82,1.43,0.00,2.35,104.86,168.54,45.45,0.00
3,109266,2994.10,9866.51,0.87,0.70,0.17,0.00,3.98,0.07,0.60,...,0.44,0.46,5.40,0.20,0.26,2.44,82.41,40.39,14.80,0.00
4,109269,9115.96,1906.30,0.04,0.00,0.00,0.04,1.85,0.00,0.10,...,0.00,0.00,3.31,0.25,0.00,2.99,175.65,87.97,18.82,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19381,142303,24470.59,2969.85,1.07,0.00,0.67,0.40,0.67,0.00,0.12,...,0.00,0.71,1.07,0.00,0.00,1.07,41.42,42.53,26.01,0.00
19382,142304,18983.82,1984.75,0.81,0.00,0.00,0.81,0.75,0.00,0.74,...,0.00,0.00,1.25,0.04,0.00,1.21,91.13,59.02,10.64,0.00
19383,142307,35129.58,4994.00,0.72,0.00,0.67,0.05,0.91,0.32,0.18,...,0.14,0.00,1.30,0.29,0.39,0.47,75.74,40.35,50.85,0.00
19384,142309,44695.97,3649.05,0.00,0.00,0.00,0.00,2.09,0.00,0.68,...,0.71,0.00,1.42,0.00,0.00,1.42,24.55,20.77,8.88,0.00


This is hopefully a nice clean DF with one row per SEQN, both DR days, and two waves lumped together.

## Add Demographics

The NHANES dataset includes demographics for each respondent. The following steps merge key demographics with each respondent id number SEQN.

Load demographic data as xpt:

In [17]:
demos_17 = pd.read_sas('data/raw/nhanes_2017_2020/P_DEMO.xpt')
demos_21 = pd.read_sas('data/raw/nhanes_2021_2023/DEMO_L.xpt')

demos_17.info()
demos_21.info()

# combine into a single dataset
demos = pd.concat([demos_17, demos_21], ignore_index=True)
demos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15560 entries, 0 to 15559
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      15560 non-null  float64
 1   SDDSRVYR  15560 non-null  float64
 2   RIDSTATR  15560 non-null  float64
 3   RIAGENDR  15560 non-null  float64
 4   RIDAGEYR  15560 non-null  float64
 5   RIDAGEMN  987 non-null    float64
 6   RIDRETH1  15560 non-null  float64
 7   RIDRETH3  15560 non-null  float64
 8   RIDEXMON  14300 non-null  float64
 9   DMDBORN4  15560 non-null  float64
 10  DMDYRUSZ  3028 non-null   float64
 11  DMDEDUC2  9232 non-null   float64
 12  DMDMARTZ  9232 non-null   float64
 13  RIDEXPRG  1874 non-null   float64
 14  SIALANG   15560 non-null  float64
 15  SIAPROXY  15560 non-null  float64
 16  SIAINTRP  15560 non-null  float64
 17  FIALANG   14481 non-null  float64
 18  FIAPROXY  14481 non-null  float64
 19  FIAINTRP  14481 non-null  float64
 20  MIALANG   11000 non-null  fl

Take the columns SEQN, age, gender, race, education, and ratio of family income to poverty:

In [18]:
demos = demos[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDEDUC2', 'INDFMPIR']]
# rename the demo columns
demos.columns = ['SEQN', 'gender', 'age', 'race', 'education', 'income_ratio']
demos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27493 entries, 0 to 27492
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SEQN          27493 non-null  float64
 1   gender        27493 non-null  float64
 2   age           27493 non-null  float64
 3   race          27493 non-null  float64
 4   education     17026 non-null  float64
 5   income_ratio  23251 non-null  float64
dtypes: float64(6)
memory usage: 1.3 MB


Merge demographics with our dietary intake data:

In [19]:
# check to see

diff = set(df['SEQN']).difference(demos['SEQN'])
print(f"{len(diff)} SEQNs missing from demographics that were in dietary intake")
diff2 = set(demos['SEQN']).difference(df['SEQN'])
print(f"{len(diff2)} SEQNs missing from dietary intake that were in demographics")

# Now join our demos with dietary intake data
df = df.merge(demos, on='SEQN', how='left')
df.info()

0 SEQNs missing from demographics that were in dietary intake
8107 SEQNs missing from dietary intake that were in demographics
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19386 entries, 0 to 19385
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SEQN                       19386 non-null  int64  
 1   weight_2d                  19386 non-null  float64
 2   grams                      19386 non-null  float64
 3   f_total_(cup_eq)           19386 non-null  float64
 4   f_citmlb_(cup_eq)          19386 non-null  float64
 5   f_other_(cup_eq)           19386 non-null  float64
 6   f_juice_(cup_eq)           19386 non-null  float64
 7   v_total_(cup_eq)           19386 non-null  float64
 8   v_drkgr_(cup_eq)           19386 non-null  float64
 9   v_redor_total_(cup_eq)     19386 non-null  float64
 10  v_redor_tomato_(cup_eq)    19386 non-null  float64
 11  v_redor_other_(cup_eq)     1938

Recode demographic variables. Coding schemes are available at the NHANES website in the documentation beside each dataset. We are splitting the income to poverty ratio into quartiles.

In [20]:
# Gender
df['gender'] = df['gender'].apply(lambda x: ('Female' if x == 2 else 'Male'))

# Education
df['education'] = df['education'].apply(
  lambda x: (
    'Less than 9th grade' if x == 1
    else '9th to 11th grade' if x == 2
    else 'High school/GED' if x == 3
    else 'Some college or AA' if x == 4
    else 'College graduate or above' if x == 5
    else "Don\'t know"
  )
)

# Race
df['race'] = df['race'].apply(
  lambda x: (
    'Mexican American' if x == 1
    else 'Other Hispanic' if x == 2
    else 'White' if x == 3
    else 'Black' if x == 4
    else 'Asian' if x == 5
    else 'Other or Multi'
  )
)

# Income to poverty ratio
df['income_ratio_qs'] = pd.qcut(
  x = df['income_ratio'],
  q = 4,
  duplicates='drop'
)
df.head()

,SEQN,weight_2d,grams,f_total_(cup_eq),f_citmlb_(cup_eq),f_other_(cup_eq),f_juice_(cup_eq),v_total_(cup_eq),v_drkgr_(cup_eq),v_redor_total_(cup_eq),...,oils_(grams),solid_fats_(grams),add_sugars_(tsp_eq),a_drinks_(no._of_drinks),gender,age,race,education,income_ratio,income_ratio_qs
0,109263,8904.03,2827.75,1.60,0.00,0.00,1.60,0.36,0.00,0.15,...,25.41,82.28,49.04,0.0,Male,2.0,Other or Multi,Don't know,4.66,"(4.22, 5.0]"
1,109264,3626.88,4177.01,0.00,0.00,0.00,0.00,4.17,0.86,1.03,...,119.77,107.89,43.70,0.0,Female,13.0,Mexican American,Don't know,0.83,"(-0.001, 1.1]"
2,109265,17806.00,4081.76,2.28,0.67,0.41,1.20,3.34,0.21,0.38,...,104.86,168.54,45.45,0.0,Male,2.0,White,Don't know,3.06,"(2.2, 4.22]"
3,109266,2994.10,9866.51,0.87,0.70,0.17,0.00,3.98,0.07,0.60,...,82.41,40.39,14.80,0.0,Female,29.0,Other or Multi,College graduate or above,5.00,"(4.22, 5.0]"
4,109269,9115.96,1906.30,0.04,0.00,0.00,0.04,1.85,0.00,0.10,...,175.65,87.97,18.82,0.0,Male,2.0,Other Hispanic,Don't know,0.96,"(-0.001, 1.1]"


## Identify plant-based protein eaters

Get the column names which identify food codes as containing any amount of
legumes, nuts/seeds, or soy. Then identify the food codes that are PBPs and label as has PBPs

In [21]:
# Keywords we will use to find column names
keywords = ['pf_legumes', 'pf_nutsds', 'pf_soy']
pbp_columns = [col for col in df.columns if any(keyword in col for keyword in keywords)]
print(pbp_columns)

# Condition: Any of the selected columns has a value greater than 1
df['has_pbp'] = (df[pbp_columns] > 1).any(axis=1)
print(df.head())

# Check that this worked
df[df['has_pbp'] == True][pbp_columns + ['has_pbp']].head(10)

# Sum the total grams of pbp in a new column
df['grams_from_pbp'] = df[pbp_columns].sum(axis=1)
print(df.head())



['pf_soy_(oz_eq)', 'pf_nutsds_(oz_eq)', 'pf_legumes_(oz_eq)']
     SEQN  weight_2d    grams  f_total_(cup_eq)  f_citmlb_(cup_eq)  \
0  109263    8904.03  2827.75              1.60               0.00   
1  109264    3626.88  4177.01              0.00               0.00   
2  109265   17806.00  4081.76              2.28               0.67   
3  109266    2994.10  9866.51              0.87               0.70   
4  109269    9115.96  1906.30              0.04               0.00   

   f_other_(cup_eq)  f_juice_(cup_eq)  v_total_(cup_eq)  v_drkgr_(cup_eq)  \
0              0.00              1.60              0.36              0.00   
1              0.00              0.00              4.17              0.86   
2              0.41              1.20              3.34              0.21   
3              0.17              0.00              3.98              0.07   
4              0.00              0.04              1.85              0.00   

   v_redor_total_(cup_eq)  ...  solid_fats_(grams)  ad

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19386 entries, 0 to 19385
Data columns (total 48 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   SEQN                       19386 non-null  int64   
 1   weight_2d                  19386 non-null  float64 
 2   grams                      19386 non-null  float64 
 3   f_total_(cup_eq)           19386 non-null  float64 
 4   f_citmlb_(cup_eq)          19386 non-null  float64 
 5   f_other_(cup_eq)           19386 non-null  float64 
 6   f_juice_(cup_eq)           19386 non-null  float64 
 7   v_total_(cup_eq)           19386 non-null  float64 
 8   v_drkgr_(cup_eq)           19386 non-null  float64 
 9   v_redor_total_(cup_eq)     19386 non-null  float64 
 10  v_redor_tomato_(cup_eq)    19386 non-null  float64 
 11  v_redor_other_(cup_eq)     19386 non-null  float64 
 12  v_starchy_total_(cup_eq)   19386 non-null  float64 
 13  v_starchy_potato_(cup_eq)  1938

## Save Data

Let's also save this as a csv so we can play around with it elsewhere:

In [23]:
df.to_csv('data/clean/df_nhanes_2017_2023.csv')